# Preprocess module

## Import Jupyter Module in other files

this code is for importing .ipynb files. YEYEEE

- code form <br/ > 
http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [4]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

- word2vec 모듈 다운로드가 요구됨

In [6]:
import format_module
import bookingreview
import preprocessreview
import wordvectormaker
import postag_module
import gensim
import xl_to_br_module
#sys.setrecursionlimit(20000)

C:\Users\Shako\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


=> Postag initiated


# 데이터 저장

In [7]:
import _pickle
import gc

def save_object_split(obj, filename, ext, split_size = 1000):
    index = 0
    for i in range(0, len(obj), split_size):
        with open(filename + "_p" + str(index) + "." + ext, 'wb') as output:
            _pickle.dump(obj[i:i+split_size], output)
            index += 1
            mem()
            gc.collect()
                
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        _pickle.dump(obj, output)

def load_object(filename):
    with open(filename, 'rb') as input:
        return _pickle.load(input)
    
def load_object_split(filename, ext, last_index):
    for index in range(last_index+1):
        with open(filename + "_p" + str(index) + "." + ext, 'rb') as input:
            if(index == 0):
                out = _pickle.load(input)
            else:
                out.extend(_pickle.load(input))
    
    return out

# Main

In [8]:
def postagModule(xl_name, reviewDB, mode = "komoran"):
    BRList = xl_to_br_module.xl_to_BookingReview(xl_name)
    
    print("adding review to DB...")
    reviewDB.add_review_list(BRList)
    
    if(mode == "twitter"):
        PRList = postag_module.twitter(BRList)
    else:
        PRList = postag_module.komoran(BRList)
        mode = "komoran"
    '''save_object(PRList_twitter, "save_PR_twitter.pkl")
    save_object(PRList_komoran, "save_PR_komoran.pkl")'''
    
    return PRList

In [9]:
def embeddingModule(PRList):
    word2vec_model = gensim.models.Word2Vec.load('./models/namuwiki_testmodel_Komoran.model')
    print("processing word embedding...")
    return wordvectormaker.word2vec_to_PreprocessReview_list(word2vec_model, PRList)

In [10]:
def formattingModule(PRList, reviewDB, version):
    print("processing formatted review...")
    formatted_list = [format_module.FormattedReview(review) for review in PRList]
    print("saving formatted review...")
    save_object(formatted_list, "./pkl/save_formatted_review_" + version +".pkl")
    return formatted_list

In [11]:
def preprocessModule(xl_name, reviewDB, version):
    PRList = postagModule(xl_name, reviewDB)
    embeddingModule(PRList)
    formatted_list = formattingModule(PRList, reviewDB, version)
    
    return formatted_list

In [12]:
reviewDB = format_module.ReviewDB("train")
format_module.FormattedReview.setDB(reviewDB)
formatted_list = preprocessModule("Commonreviews_snuproject _test.xlsx", reviewDB, "test")

processing 0 th unit...
adding review to DB...
postagging [komoran] 0 ith unit...
processing word embedding...

tokenized : 좋겟 어여
stemmed :  ['좋다', '어여']
postagged : ['좋/VA', '다/EC', '어/NNG', '이/VCP', '어/EC']
좋겟어여/NA not in vocabulary so replaced to ['좋/VA', '다/EC', '어/NNG', '이/VCP', '어/EC']

tokenized : ㅋ ㅋ 신림 점
stemmed :  ['ㅋ', 'ㅋ', '신림', '점']
postagged : ['ㅋ/NA', 'ㅋ/NA', '신림/NNP', '점/NNB']
ㅋㅋ신림점/NA not in vocabulary so replaced to ['ㅋ/NA', 'ㅋ/NA', '신림/NNP', '점/NNB']

tokenized : ㅋ ㅋ ㅋ ㅋ 왘 ㅋ ㅋ ㅋ 평가 보소
stemmed :  ['ㅋ', 'ㅋ', 'ㅋ', 'ㅋ', '왘', 'ㅋ', 'ㅋ', 'ㅋ', '평가', '보다']
postagged : ['ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', '왘/NA', 'ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', '평가/NNG', '보다/MAG']
ㅋㅋㅋㅋ왘ㅋㅋㅋ평가보소/NA not in vocabulary so replaced to ['ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', '왘/NA', 'ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', '평가/NNG', '보다/MAG']

tokenized : ㅋ ㅋ ㅋ 이 근처 굽네 가 하나 라 그나마 밥벌어 먹고 사나 본데
stemmed :  ['ㅋ', 'ㅋ', 'ㅋ', '이', '근처', '굽', '네', '가다', '하나', '라', '그나마', '밥', '벌다', '먹다', '사나', '보', '인데']
postagged : ['ㅋ/NA', 'ㅋ/NA', 'ㅋ/NA', '이

무표정..ㅎㄷㄷ/NA not in vocabulary so replaced to ['무/XPN', '표정/NNG']

tokenized : 왜 이러 지 ㅠ ㅠ
stemmed :  ['왜', '이르다', '지다', 'ㅠ', 'ㅠ']
postagged : ['왜/MAG', '이르/VV', '다/EC', '지/VX', '다/EC', 'ㅠ/NA', 'ㅠ/NA']
왜이러지ㅠㅠ/NA not in vocabulary so replaced to ['왜/MAG', '이르/VV', '다/EC', '지/VX', '다/EC', 'ㅠ/NA', 'ㅠ/NA']

tokenized : 젊은
stemmed :  ['젊다']
postagged : ['젊/VA', '다/EC']
젊은 남자/NNP not in vocabulary so replaced to ['젊/VA', '다/EC']

tokenized : 질겨요 ㅜ ㅜ
stemmed :  ['질겨', '요', 'ㅜ', 'ㅜ']
postagged : ['질/NNG', '겨/NNG', '요/MM', 'ㅜ/NA', 'ㅜ/NA']
질겨요ㅜㅜ/NA not in vocabulary so replaced to ['질/NNG', '겨/NNG', '요/MM', 'ㅜ/NA', 'ㅜ/NA']

tokenized : 인듯 해요 ㅜ ㅜ
stemmed :  ['이다', '해', '요', 'ㅜ', 'ㅜ']
postagged : ['이/NP', '다/JX', '하/VV', '아/EC', '요/MM', 'ㅜ/NA', 'ㅜ/NA']
인듯해요ㅜㅜ/NA not in vocabulary so replaced to ['이/NP', '다/JX', '하/VV', '아/EC', '요/MM', 'ㅜ/NA', 'ㅜ/NA']

tokenized : 친구 랑 두잔 씩 먹고 옴 ㅋ ㅋ
stemmed :  ['친구', '랑', '두', '잔', '씩', '먹다', '옴', 'ㅋ', 'ㅋ']
postagged : ['친구/NNG', '랑/NNP', '두/MM', '자/VV', 'ㄴ/ETM', '씩/

postagged : ['조/NR', '은/JX', '추억/NNG', '만들/VV', '다/EC', '고갑니닼/NA', 'ㅋ/NA']
조은추억만들고갑니닼ㅋ/NA not in vocabulary so replaced to ['조/NR', '은/JX', '추억/NNG', '만들/VV', '다/EC', 'ㅋ/NA']

tokenized : 안됬 으면
stemmed :  ['안', '되다']
postagged : ['안/MAG', '되/VV', '다/EC']
안됬으면/NA not in vocabulary so replaced to ['안/MAG', '되/VV', '다/EC']

tokenized : ㅠ 슈
stemmed :  ['ㅠ', '슈']
postagged : ['ㅠ/NA', '슈/EC']
ㅠ슈/NA not in vocabulary so replaced to ['ㅠ/NA', '슈/EC']

tokenized : 맛있 었으뮤
stemmed :  ['맛있다', '었으뮤']
postagged : ['맛있/VA', '다/EC', '었으뮤/NA']
맛있었으뮤/NA not in vocabulary so replaced to ['맛있/VA', '다/EC']

tokenized : 멕시 카나 치킨
stemmed :  ['멕시', '카나', '치킨']
postagged : ['멕시/NA', '카/NNG', '나/JC', '치킨/NNG']
멕시카나치킨/NA not in vocabulary so replaced to ['카/NNG', '나/JC', '치킨/NNG']

tokenized : 처음 밧어요
stemmed :  ['처음', '밧어', '요']
postagged : ['처음/NNG', '밧어/NA', '요/MM']
처음밧어요/NA not in vocabulary so replaced to ['처음/NNG', '요/MM']

tokenized : 쭌 곱창
stemmed :  ['쭌', '곱창']
postagged : ['쭌/NA', '곱창/NNP']
쭌곱창/NA not in 

stemmed :  ['02', '3', '36', '20', '01']
postagged : ['02/SN', '3/SN', '36/SN', '20/SN', '01/SN']
023362001/SN not in vocabulary so replaced to ['02/SN', '3/SN', '36/SN', '20/SN', '01/SN']

tokenized : go go go
stemmed :  ['go', 'go', 'go']
postagged : ['go/SL', 'go/SL', 'go/SL']
gogogo/SL not in vocabulary so replaced to ['go/SL', 'go/SL', 'go/SL']

tokenized : 플래 이팅
stemmed :  ['플', '래', '이팅']
postagged : ['프/VX', 'ㄹ/ETM', '래/NNG', '이팅/NA']
플래이팅/NA not in vocabulary so replaced to ['프/VX', 'ㄹ/ETM', '래/NNG', '이팅/NA']

tokenized : 엄청 재미 있네요 ㅡ
stemmed :  ['엄청', '재미', '있다', 'ㅡ']
postagged : ['엄청/MAG', '재미/NNG', '있/VV', '다/EC', 'ㅡ/NA']
엄청재미있네요ㅡ/NA not in vocabulary so replaced to ['엄청/MAG', '재미/NNG', '있/VV', '다/EC', 'ㅡ/NA']

tokenized : 유머 있는 말과센슼 ㅋ
stemmed :  ['유머', '있다', '말과센슼', 'ㅋ']
postagged : ['유머/NNG', '있/VV', '다/EC', '말과센슼/NA', 'ㅋ/NA']
유머있는말과센슼ㅋ/NA not in vocabulary so replaced to ['유머/NNG', '있/VV', '다/EC', 'ㅋ/NA']

tokenized : 시간 가는 지 몰랐 네요 ㅡ
stemmed :  ['시간', '가다', '지다', '몰르다', '

postagged : ['맛있/VA', '다/EC', '네욯/NA']
맛있네욯/NA not in vocabulary so replaced to ['맛있/VA', '다/EC']

tokenized : 주고 완전 강츄
stemmed :  ['주다', '완전', '강츄']
postagged : ['주/VX', '다/EC', '완전/NNG', '강츄/NA']
주고완전강츄/NA not in vocabulary so replaced to ['주/VX', '다/EC', '완전/NNG']

tokenized : ㅇ ㅇ 솔직히
stemmed :  ['ㅇ', 'ㅇ', '솔직하다']
postagged : ['ㅇ/NA', 'ㅇ/NA', '솔직/XR', '하/XSA', '다/EC']
ㅇㅇ솔직히/NA not in vocabulary so replaced to ['ㅇ/NA', 'ㅇ/NA', '솔직/XR', '하/XSA', '다/EC']

tokenized : 높은 퀄리티 에
stemmed :  ['높다', '퀄리티', '에']
postagged : ['높/VA', '다/EC', '퀄리티/NA', '에/JKB']
높은퀄리티에/NA not in vocabulary so replaced to ['높/VA', '다/EC', '퀄리티/NA', '에/JKB']

tokenized : GRENADIER
stemmed :  ['GRENADIER']
postagged : ['GRENADIER/SL']
GRENADIER/SL not in vocabulary so replaced to []

tokenized : 사 먹어 봤거든요
stemmed :  ['사', '먹다', '보다']
postagged : ['사/NNG', '먹/VV', '다/EC', '보다/MAG']
사먹어봤거든요!ㅋㅋㅋ/NA not in vocabulary so replaced to ['사/NNG', '먹/VV', '다/EC', '보다/MAG']

tokenized : 시카고 피자
stemmed :  ['시카고', '피자']
postagg

In [5]:
#reviewDB = format_module.ReviewDB("validation")
reviewDB = format_module.ReviewDB("train")
print(reviewDB.review_dict[41099].next)

ReviewNode - BookingReview object:
  Company name = 공차 신림역점
  ID = 0___****
  Rating = 10
  Context = 여기 종종 이용하는데 전 항상 친절해서 좋앗는데... 대체적으로 평이 안좋네요  여기 남자분 알바 중에 영웅재중?닮은 키크고 눈 좀 서납게 생기신분 잇는데 항상 갈때마다 음료추천도 잘해주시고 귀찮게 이것저것 물어봐도 웃으시면서 친절하게 응대해주시는게 넘 감사해서 그분 칭찬해드려여... 그 알바분은 회사에서 상 줫으면 좋겟어여 ㅋㅋ신림점 진상손님도 많을텐데 갈때마다 친절해서 항상 기분좋앗어여
  Post time = 41999.687138217596
  Label = False
  Review id = 40093




In [16]:
a = load_object("./pkl/save_formatted_review_train.pkl")
print(a[0])

FormattedReview object 41099:
  context = 
[[ 0.13338269  2.57921386  1.10160124 ...,  0.86103064  2.78854179
   1.87262499]
 [-0.13127854  1.65661263 -0.70222795 ...,  1.08186483  1.34531033
   3.25771809]
 [ 1.09225094  0.87828368  3.05598545 ...,  3.32895207 -0.73563319
  -0.32933012]
 ..., 
 [-4.64829493  5.1486454  -2.75840378 ...,  3.04924154  0.15255213
  -1.48532724]
 [ 1.62699354  1.45267355 -0.51794469 ..., -1.59229398  3.52136612
   1.01794887]
 [ 3.2266624   0.68326694 -0.93881059 ..., -1.524984    1.03987861
  -1.45865238]]
  context_bayes = 
['동네/NNG', '맛집/NNG', '이/VCP', '라/EC', '찾/VV', '았/EP', '는데/EC', '고급/NNG', '스럽/XSA', 'ㄴ/ETM', '한/MM', '정식/NNG', '집/NNG', '에/JKB', '서비스/NNG', '도/JX', '너무/MAG', '친절/NNG', '하/XSV', '아서/EC', '반하/VV', '았/EP', '어요/EC', '가격/NNG', '대비/NNG', '음식/NNG', '도/JX', '푸짐/XR', '하/XSA', '고/EC', '맛있/VA', '어서/EC', '즐겁/VA', 'ㄴ/ETM', '시간/NNG', '보내/VV', '었/EP', '습/NNG', '이/VCP', '니/EC', '당/XSN', '!/SF']
  comp_similarity = 0.5402818532208692
  rate = 1.0
  rei

In [ ]:
print(formatted_list[0].context.shape)
print(formatted_list[1].context.shape)
print(formatted_list[2].context.shape)

In [ ]:
import bookingreview
import format_module

reviewDB = format_module.ReviewDB()
reviewList = [bookingreview.BookingReview("회사", "id", 10, "asdfsadf", 40000.236+i*0.001, True, i) for i in range(50000)]

reviewDB.add_review_list(reviewList)
reviewDB.size()